---
jupytext:
  cell_metadata_filter: all, -hidden, -heading_collapsed, -run_control, -trusted
  notebook_metadata_filter: all, -jupytext.text_representation.jupytext_version, -jupytext.text_representation.format_version, -language_info.version, -language_info.codemirror_mode.version, -language_info.codemirror_mode, -language_info.file_extension, -language_info.mimetype, -toc
  text_representation:
    extension: .md
    format_name: myst
kernelspec:
  display_name: Python 3 (ipykernel)
  language: python
  name: python3
language_info:
  name: python
  nbconvert_exporter: python
  pygments_lexer: ipython3
nbhosting:
  title: 'Correction du TP régression Logistique'
  version: '1.0'
---

<div class="licence">
<span><img src="media/logo_IPParis.png" /></span>
<span>Lisa BEDIN<br />Pierre André CORNILLON<br />Eric MATZNER-LOBER</span>
<span>Licence CC BY-NC-ND</span>
</div>

# Modules python

Importer les modules pandas (comme `pd`) numpy (commme `np`) matplotlib.pyplot (comme `plt`) et statsmodels.formula.api (comme `smf`).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# Régression logistique


## Importation des données

Importer les données `artere.txt` dans le DataFrame pandas `artere` \[`read_csv` de `numpy` \]. Sur Fun Campus le chemin est `data/artere.txt`. Outre l'age et la présence=1/absence=0 de la maladie cardio-vasculaire (`chd`) une variable qualitative à 8 modalités donne les classes d'age (`agegrp`)

In [ ]:
artere = pd.read_csv("data/artere.txt", header=0, sep=" ")

## Nuage de points

Tracer le nuage de points avec `age` en abscisses et `chd` en ordonnées \[`plt.plot` \]

In [ ]:
plt.scatter(x="age", y="chd", c="chd", data=artere)
plt.show()

## Régression logistique

Effectuer une régression logistique où `age` est la variable explicative et `chd` la variable binaire à expliquer. Stocker le résultat dans l'objet `modele` et

1.  effectuer le résumé de cette modélisation;

In [ ]:
```{code-cell} python
modele = smf.logit('chd~age', data=artere).fit()
print(modele.summary())
```

Le modèle possède une log-vraisemblance de -53.677, le modèle avec uniquement la constante (appelé ici Null) à de son coté une log-vraisemblance de -68.331. L'ajout de l'âge permet un gain de vraisemblance assez conséquent.
2.  afficher l'attribut contenant les paramètres estimés par régression logistique.

```{code-cell} python
print(modele.params)
```


## Prévision et probabilités estimées

Afficher l'ajustement/prévision pour les données de l'échantillon via la méthode `predict` (sans arguments) sur le modèle `reg`. Que représente ce vecteur:

-   une probabilité d'être malade pour chaque valeur de l'age de l'échantillon (OUI on modélise la probabilité que $Y=1$ et dans l'échantillon $Y_i=1$ équivaut à $Y_i$ malade)
-   une probabilité d'être non-malade pour chaque valeur de l'age de l'échantillon (NON)
-   une prévision de l'état malade/non-malade pour chaque valeur de l'age de l'échantillon (NON, par défaut la fonction renvoie la probabilité)

In [ ]:
modele.predict()

Donner la prévision de l'état malade/non-malade avec l'indicatrice que $\hat p(x)>s$ où $s$ est le seuil classique de 0.5.

In [ ]:
print(modele.predict()>0.5)

# Simulation de données  Variabilité de $\hat \beta_2$


## Simulation

1.  Générer $n=100$ valeurs de $X$ uniformément entre 0 et 1.
2.  Pour chaque valeur $X_i$ simuler $Y_i$ selon un modèle logistique de paramètres $\beta_1=-5$ et $\beta_2=10$

\[créer une instance de générateur aléatoire `np.random.default_rng` et utiliser `rng.uniform` et `rng.binomial` \]

In [ ]:
rng = np.random.default_rng(seed=123)
X = rng.uniform(size=100)
Y = np.copy(X)
for i,xi in enumerate(X):
    proba = 1 / (1 + np.exp( -(-5 + 10 * xi) ))
    Y[i]=rng.binomial(1, proba, 1)[0]
df = pd.DataFrame({"X" : X, "Y" : Y})

## Estimation

Estimer les paramètres $\beta_1$ et $\beta_2$

In [ ]:
modele = smf.logit('Y~X', data=df).fit()
print(modele.params)

## Variabilité de l'estimation

Refaire les deux questions ci-dessus 500 fois et constater par un graphique adapté la variabilité de $\hat \beta_2$.

Simulons 500 fois les données

In [ ]:
hatbeta2 = []
for it in range(500):
    X = rng.uniform(size=100)
    Y = np.copy(X)
    for i,xi in enumerate(X):
        proba = 1 / (1 + np.exp( -(-5 + 10 * xi) ))
        Y[i]=rng.binomial(1, proba, 1)[0]
    df = pd.DataFrame({"X" : X, "Y" : Y})
    modele = smf.logit('Y~X', data=df).fit()
    hatbeta2.append(modele.params[1])

Et construisons un histogramme

In [ ]:
plt.hist(hatbeta2, bins=30)
plt.show()

# Deux régressions logistiques simples


## Importation des données

Importer les données `artere.txt` dans le DataFrame pandas `artere` \[`read_csv` de `numpy` \]. Sur Fun Campus le chemin est `data/artere.txt`. Outre l'age et la présence=1/absence=0 de la maladie cardio-vasculaire (`chd`) une variable qualitative à 8 modalités donne les classes d'age (`agegrp`)

In [ ]:
artere = pd.read_csv("data/artere.txt", header=0, sep=" ")

## Deux régressions logistiques

1.  Effectuer une régression logistique simple où `age` est la variable explicative et `chd` la variable binaire à expliquer;
2.  Refaire la même chose avec la racine carrée de `age` comme variable explicative;

In [ ]:
modele1 = smf.logit('chd~age', data=artere).fit()
modele2 = smf.logit('chd~I(np.sqrt(age))', data=artere).fit()

## Comparaison

Ajouter au nuage de points les 2 ajustements (la droite et la "racine carrée") et choisir le meilleur modèle via un critère numérique. \[méthode `argsort` sur une colonne du DataFrame et `plt.plot` ; utiliser le résumé des modèles\]

In [ ]:
sel = artere['age'].argsort()
plt.scatter(x="age", y="chd", c="chd", data=artere)
plt.plot(artere.age.iloc[sel], modele1.predict()[sel], "b-", artere.age.iloc[sel], modele2.predict()[sel], "r-"  )
plt.show()

Comme les deux modèles ont le même nombre de variables explicatives nous pouvons comparer les log-vraisemblances et la plus élevée donne le meilleur modèle. C'est le modèle 1 qui l'emporte mais les log-vraisemblances sont assez comparables.